In [52]:
%pip install scikit-surprise

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 11.8 MB 2.2 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1623195 sha256=423700414e7e8dd7be81c998b4df90a9e2f740427bb065641d919c3122823031
  Stored in directory: /tmp/xdg_cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [1]:
%pip install -q kaggle

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.1.1 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.1.1 which is incompatible.
moto 1.3.14 requires idna<2.9,>=2.5, but you have idna 2.10 which is incompatible.
aws-sam-translator 1.38.0 requires jsonschema~=3.2, but you have jsonschema 4.4.0 which is incompatible.
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [3]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download -d grouplens/movielens-20m-dataset

 99%|███████████████████████████████████████▋| 194M/195M [00:04<00:00, 74.3MB/s]
100%|████████████████████████████████████████| 195M/195M [00:05<00:00, 34.9MB/s]


In [5]:
 ! unzip movielens-20m-dataset.zip -d dataset

Archive:  movielens-20m-dataset.zip
  inflating: dataset/genome_scores.csv  
  inflating: dataset/genome_tags.csv  
  inflating: dataset/link.csv        
  inflating: dataset/movie.csv       
  inflating: dataset/rating.csv      
  inflating: dataset/tag.csv         


# Train/test division

In [6]:
import numpy as np
import pandas as pd

In [7]:
ratings = pd.read_csv('dataset/rating.csv')

In [8]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [9]:
users = ratings['userId'].unique() #list of all users
movies = ratings['movieId'].unique() #list of all movies

In [10]:
test = pd.DataFrame(columns=ratings.columns)
train = pd.DataFrame(columns=ratings.columns)

test_ratio = 0.2

In [11]:
users_train = []
users_test = []

In [16]:
# We can use info about timestamp to split data

i = 0
for u in users:
    user_reviews = ratings[ratings['userId'] == u]
    n = len(user_reviews)
    test_size = int(test_ratio * n)

    user_reviews = user_reviews.sort_values('timestamp').reset_index()
    user_reviews.drop('index', axis=1, inplace=True)
        
    user_test = user_reviews.iloc[n - 1 - test_size :]
    user_train = user_reviews.iloc[: n - 2 - test_size]
     
    users_train.append(user_train)
    users_test.append(user_test)

In [17]:
test = pd.concat(users_test)

In [18]:
train = pd.concat(users_train)

In [39]:
train.to_csv('mov_train.csv', index=False)

In [40]:
test.to_csv('mov_test.csv', index=False)

In [ ]:
train.to_csv('train_sorted.csv')
test.to_csv('test_sorted.csv')

In [21]:
train

,userId,movieId,rating,timestamp
0,1,924,3.5,2004-09-10 03:06:38
1,1,919,3.5,2004-09-10 03:07:01
2,1,2683,3.5,2004-09-10 03:07:30
3,1,1584,3.5,2004-09-10 03:07:36
4,1,1079,4.0,2004-09-10 03:07:45
...,...,...,...,...
292,138493,1955,4.0,2009-10-28 17:32:34
293,138493,765,4.0,2009-10-29 22:19:21
294,138493,5785,3.5,2009-10-29 22:19:35
295,138493,1429,4.0,2009-10-29 22:20:05


In [22]:
test

,userId,movieId,rating,timestamp
139,1,5540,4.0,2005-04-02 23:51:45
140,1,7164,3.5,2005-04-02 23:52:03
141,1,2021,4.0,2005-04-02 23:52:09
142,1,7046,4.0,2005-04-02 23:52:14
143,1,2143,4.0,2005-04-02 23:52:31
...,...,...,...,...
368,138493,6534,3.0,2009-12-07 18:18:28
369,138493,53464,4.0,2009-12-07 18:18:40
370,138493,1275,3.0,2010-01-01 20:42:32
371,138493,6996,3.0,2010-01-01 20:42:35


In [28]:
n_users = len(users)
n_movies = len(movies)

In [29]:
n_users, n_movies

(138493, 26744)

In [45]:
train.drop('timestamp', axis=1, inplace=True)
test.drop('timestamp', axis=1, inplace=True)

In [46]:
# Renumbering, becouse we have gaps

train['userId'] = train['userId'].map({u: i for i, u in enumerate(users)})
test['userId'] = test['userId'].map({u: i for i, u in enumerate(users)})
train['movieId'] = train['movieId'].map({m: i for i, m in enumerate(movies)})
test['movieId'] = test['movieId'].map({m: i for i, m in enumerate(movies)})

In [47]:
train

,userId,movieId,rating
0,0,20,3.5
1,0,19,3.5
2,0,86,3.5
3,0,61,3.5
4,0,23,4.0
...,...,...,...
292,138492,1244,4.0
293,138492,3569,4.0
294,138492,1750,3.5
295,138492,2634,4.0


In [48]:
test

,userId,movieId,rating
139,0,137,4.0
140,0,159,3.5
141,0,68,4.0
142,0,157,4.0
143,0,73,4.0
...,...,...,...
368,138492,832,3.0
369,138492,972,4.0
370,138492,2233,3.0
371,138492,847,3.0


# SVD

In [53]:
from surprise.prediction_algorithms import SVD
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from surprise.model_selection import GridSearchCV

In [106]:
reader = Reader(rating_scale=(train.rating.min(), train.rating.max()))

In [57]:
train_sp = Dataset.load_from_df(train, reader)

In [58]:
VALIDATION_FRAC = 0.05

In [59]:
train_set, valid_set = train_test_split(train_sp, train_size=1 - VALIDATION_FRAC, test_size=VALIDATION_FRAC)

In [60]:
svd = SVD(n_factors=50, n_epochs=10, lr_all=0.05)

In [61]:
svd.fit(train_set)

In [62]:
preds = svd.test(valid_set)

In [63]:
y_pred = []
y_true = []
for i in range(len(preds)):
    y_pred.append(preds[i].est)
    y_true.append(preds[i].r_ui)

In [65]:
len(y_pred)

788835

In [88]:
from sklearn.metrics import r2_score, accuracy_score, mean_squared_error, mean_absolute_error

In [68]:
print(f'r2_score: {r2_score(y_true, y_pred)}')
print(f'mean_squared_error: {mean_squared_error(y_true, y_pred)}')

r2_score: 0.298508586649587
mean_squared_error: 0.7788608616987218


In [71]:

param_grid = {
    'n_epochs': [10], 
    'lr_all': [0.005, 0.01, 0.05, 0.1], 
    'reg_all': [0.4, 0.6], 
    'n_factors': [10, 30, 50]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(train_sp)

# best RMSE score
print(gs.best_score['rmse'])

# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

0.8842877652305642
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 10}


In [72]:
print(gs.best_score['mae'])

print(gs.best_params['mae'])

0.6866013861982388
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 10}


In [73]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8842877652305642
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 10}


In [78]:
param_grid = {
    'n_epochs': [10], 
    'lr_all': [0.005], 
    'reg_all': [0.4], 
    'n_factors': [5, 10, 20]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(train_sp)

print('rmse:')
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print()
print('mae:')
print(gs.best_score['mae'])
print(gs.best_params['mae'])

rmse:
0.8841479041085282
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 5}

mae:
0.6862865244840086
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 5}


In [79]:
param_grid = {
    'n_epochs': [10, 20], 
    'lr_all': [0.001, 0.005], 
    'reg_all': [0.4], 
    'n_factors': [5, 10, 20]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(train_sp)

print('rmse:')
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print('')
print('mae:')
print(gs.best_score['mae'])
print(gs.best_params['mae'])

rmse:
0.8831234949109747
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 10}

mae:
0.6855297761300689
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 10}


In [80]:
param_grid = {
    'n_epochs': [10], 
    'lr_all': [0.005], 
    'reg_all': [0.4], 
    'n_factors': [1, 2, 3, 5]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(train_sp)

print('rmse:')
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print()
print('mae:')
print(gs.best_score['mae'])
print(gs.best_params['mae'])

rmse:
0.8843523557238782
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 1}

mae:
0.6864876134000771
{'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 1}


In [ ]:
# Params are same in mae and mse best models

In [90]:
algo = gs.best_estimator['rmse']
algo.fit(train_sp.build_full_trainset())

In [92]:
print(f'best rmse: {gs.best_score["rmse"]}')
print(f'params: {gs.best_params["rmse"]}')

preds = algo.test(valid_set)
y_pred = []
y_true = []
for i in range(len(preds)):
    y_pred.append(preds[i].est)
    y_true.append(preds[i].r_ui)
print(f'r2_score: {r2_score(y_true, y_pred)}')
print(f'mean_squared_error: {mean_squared_error(y_true, y_pred)}')
print(f'rmse: {mean_squared_error(y_true, y_pred, squared=False)}')
print(f'mae: {mean_absolute_error(y_true, y_pred)}')

best rmse: 0.8843523557238782
params: {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 1}
r2_score: 0.31348423955243054
mean_squared_error: 0.7622335021866485
rmse: 0.8730598502889986
mae: 0.6785521234794458


In [98]:
param_grid = {
    'n_epochs': [20], 
    'lr_all': [0.005], 
    'reg_all': [0.4], 
    'n_factors': [10, 15]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(train_sp)

print('rmse:')
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])
print('')
print('mae:')
print(gs.best_score['mae'])
print(gs.best_params['mae'])

rmse:
0.8833191159124283
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 10}

mae:
0.6854937175264896
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 10}


In [ ]:
# Params are same in mae and mse best models

In [99]:
algo = gs.best_estimator['rmse']
algo.fit(train_sp.build_full_trainset())

print(f'best rmse: {gs.best_score["rmse"]}')
print(f'params: {gs.best_params["rmse"]}')

preds = algo.test(valid_set)
y_pred = []
y_true = []
for i in range(len(preds)):
    y_pred.append(preds[i].est)
    y_true.append(preds[i].r_ui)
print(f'r2_score: {r2_score(y_true, y_pred)}')
print(f'mean_squared_error: {mean_squared_error(y_true, y_pred)}')
print(f'rmse: {mean_squared_error(y_true, y_pred, squared=False)}')
print(f'mae: {mean_absolute_error(y_true, y_pred)}')

best rmse: 0.8833191159124283
params: {'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.4, 'n_factors': 10}
r2_score: 0.3153040677941975
mean_squared_error: 0.7602129600024518
rmse: 0.8719019210911579
mae: 0.6775071852527216


In [109]:
test_sp = Dataset.load_from_df(test, reader)

In [102]:
# https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py

In [116]:
_, test_set = train_test_split(test_sp, test_size=0.9999999)

In [117]:
from collections import defaultdict

In [118]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

predictions = algo.test(test_set)
precisions, recalls = precision_recall_at_k(predictions, k=10, threshold=4)

    # Precision and recall can then be averaged over all users
print(sum(prec for prec in precisions.values()) / len(precisions))
print(sum(rec for rec in recalls.values()) / len(recalls))

0.40991636560855693
0.1721750243098804
